# Oxylabs Search


This notebook goes over how to use the Oxylabs search component.

First, you need to set up the proper API keys and environment variables. Create your API user credentials: Sign up for a free trial or purchase the product in the [Oxylabs dashboard](https://dashboard.oxylabs.io/en/registration) to create your API user credentials (OXYLABS_USERNAME and OXYLABS_PASSWORD).



## Setup
The integration lives in the `langchain-community` package.

In [1]:
# %pip install -U langchain-community oxylabs

In [2]:
import getpass
import os
from pprint import pprint

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

## Basic usage

In [3]:
from langchain_community.utilities import OxylabsSearchAPIWrapper

search = OxylabsSearchAPIWrapper()

In [4]:
pprint(search.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '    IMAGES-ITEM-4: Redacted base64 image string...\n'
 '    IMAGES-ITEM-5: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?num=5&sca_esv=e02818271ce7f460&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwiq-7XO3L2JAxUVp68BHR7wFowQmxMoAHoECDAQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4

## Number of results
You may use the parameters `start_page`, `pages` and `limit` to specify the number of results. The default values are:  
    * `start_page = 1`  
    * `pages = 1`  
    * `limit = 5`  

In [5]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)


In [6]:
pprint(search.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '    IMAGES-ITEM-4: Redacted base64 image string...\n'
 '    IMAGES-ITEM-5: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?sca_esv=e02818271ce7f460&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwiy7vPY3L2JAxVUhYkEHV9LIBkQmxMoAHoECDsQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4: \n'


## Image Content Display
You may use the `include_binary_image_data` parameter to manage the display of binary image data. By default, this parameter is set to `False`, and the image data is replaced with the text "Redacted base64 image string...". 

In [7]:
search = OxylabsSearchAPIWrapper(
    include_binary_image_data=True,
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)
pprint(search.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: '
 '/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIALoAugMBEQACEQEDEQH/xAAbAAEBAAIDAQAAAAAAAAAAAAAAAQUHAgQGA//EAEcQAAIBAgIECAgLBgcBAAAAAAABAgMEBREGEiExBxNBUVNhcZEWIlKBkqHB0RQVMjZVcnOUsbLSMzVCgpOzIzRDVGLw8Rf/xAAbAQEBAAMBAQEAAAAAAAAAAAAAAQMEBQYCB//EADERAQACAQIEAwYGAwEBAAAAAAABAgMEERIhMVEFExQyM0FScYEVImGRobHB0eE0Qv/aAAwDAQACEQMRAD8A3iAAAAAADjKSSzzWwDymN6c4bh8pUbRSva62NU2lBPrl7kzax6S9+c8mrk1dK8o5vJ3unuNXDaoujbRfJThm152bVdHjjrzattXknpyYW4x7F7j9rid4/q1nH1RyM8YscdKwwzlyT1tLp1Lm4qbalxXn9eo2fUREdIfPFb4vk9+be3tLKOcLivTedOvVh9Wo0TaJ6rEzHR26GOYtbv8AwcTvI9taUl3PMxzjpPWIfUZLx0mf3Zix09xy11VVqUrmK38bBJvzrIxW0uOenJnrqskfq9Zg3CDht7NUr6nOxqvdKT1qb/m5POjVvpb15xzbOPVUtytyevpyjOKlBpxe1NPYzVbTmAAAAAAAAAAAAACPcBrXhA0kq1rqeE2dSUKNLZcSg9tSXk58yOjpMEcPHZztVmmbcFX

## Result Filtering / Ordering

### Default result group order :  
* `knowledge_graph`  
* `combined_search_result`  
* `product_information`  
* `local_information`  
* `search_information`  
  
You may want to adjust the display order of result groups and exclude the `product_information`, `local_information`  and `combined_search_result` groups in cases where only abstract knowledge is needed. For example, this configuration allows tailored result presentation.

In [8]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 1,
        "limit": 10,
        "geo_location": "Kairo,Egypt",
        "result_categories": [
            "search_information",
            "knowledge_graph",            
        ]
    }
)
pprint(search.run("Visiting Great Pyramid of Giza in Egypt"), indent=4)


('  SEARCH INFORMATION: \n'
 '  QUERY: Visiting Great Pyramid of Giza in Egypt\n'
 '  GEO_LOCATION: Cairo, Egypt\n'
 '  SHOWING_RESULTS_FOR: Visiting Great Pyramid of Giza in Egypt\n'
 '  TOTAL_RESULTS_COUNT: 6480000\n'
 '\n'
 '  RELATED SEARCHES: \n'
 '  RELATED_SEARCHES ITEMS: \n'
 '    RELATED_SEARCHES-ITEM-1: Visiting great pyramid of giza in egypt cost\n'
 '    RELATED_SEARCHES-ITEM-2: Pyramids of Giza entrance fee\n'
 '    RELATED_SEARCHES-ITEM-3: Best time to visit pyramids\n'
 '    RELATED_SEARCHES-ITEM-4: Visiting the pyramids without a guide\n'
 '    RELATED_SEARCHES-ITEM-5: Where to stay in Egypt to see pyramids\n'
 '    RELATED_SEARCHES-ITEM-6: Giza Pyramids official website\n'
 '    RELATED_SEARCHES-ITEM-7: Can you visit the Pyramids from Sharm El '
 'Sheikh\n'
 '    RELATED_SEARCHES-ITEM-8: Giza Pyramids tickets office and entrance\n'
 '\n'
 '  RELATED QUESTIONS: \n'
 '  ITEMS ITEMS: \n'
 '    ITEMS-ITEM-1: \n'
 '    POS: 1\n'
 '    ANSWER: The interiors of all three pyra

You may want to adjust the display order of result groups and exclude abstract knowledge when focusing specifically on products and local services. In the following example reorders result groups to prioritise search information and local services, omitting abstract knowledge to enhance relevance for this context.

In [9]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 5,
        "limit": 15,
        "geo_location": "Belgium",
        "result_categories": [
            "search_information",
            "local_information",            
        ]
    }
)
pprint(search.run("Open Working Space in Belgium."))

('  SEARCH INFORMATION: \n'
 '  QUERY: Open Working Space in Belgium.\n'
 '  GEO_LOCATION: 1420, Eigenbrakel\n'
 '  SHOWING_RESULTS_FOR: Open Working Space in Belgium.\n'
 '  TOTAL_RESULTS_COUNT: 173000000\n'
 '  NO_RESULTS_FOR_ORIGINAL_QUERY_FOUND: True\n'
 '\n'
 '  RELATED SEARCHES: \n'
 '  RELATED_SEARCHES ITEMS: \n'
 '    RELATED_SEARCHES-ITEM-1: Coworking Antwerpen\n'
 '    RELATED_SEARCHES-ITEM-2: Firma II\n'
 '    RELATED_SEARCHES-ITEM-3: Spaces Tour & Taxis\n'
 '    RELATED_SEARCHES-ITEM-4: Silversquare\n'
 '\n'
 '  LOCAL PACK: \n'
 '  ITEMS ITEMS: \n'
 '    ITEMS-ITEM-1: \n'
 '    POS: 1\n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: http://www.betacowork.com/\n'
 '      TITLE: Website\n'
 '      LINKS-ITEM-2: \n'
 '      HREF: '
 '/maps/dir//Betacowork,+Witte+Patersstraat+4,+1040+Brussel/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x47c3c4b5c22df6af:0xa4ef418da17d1e1a?sa=X&ved=1t:57443&hl=nl&gl=be&ictx=111\n'
 '      TITLE: Route\n'
 '    PHONE: 0472 09 38 70\n'
 '    

## Raw Results

The wrapper method `results()` returns all raw results in JSON format.

In [10]:
search = OxylabsSearchAPIWrapper()

In [11]:
search.results("Python Programming Language")

[{'paid': [],
  'organic': [{'pos': 1,
    'url': 'https://www.python.org/',
    'desc': 'The official home of the Python Programming Language.',
    'title': 'Welcome to Python.org',
    'sitelinks': {'expanded': [{'url': 'https://www.python.org/about/gettingstarted/',
       'title': 'Python For Beginners'},
      {'url': 'https://docs.python.org/3/tutorial/',
       'title': 'The Python Tutorial'},
      {'url': 'https://www.python.org/downloads/', 'title': 'Downloads'},
      {'url': 'https://www.python.org/about/', 'title': 'About'},
      {'url': 'https://www.python.org/downloads/windows/',
       'title': 'Python Releases for Windows'}]},
    'url_shown': 'https://www.python.org',
    'pos_overall': 1,
    'favicon_text': 'Python.org'},
   {'pos': 2,
    'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)',
    'desc': 'Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant inden

## Tool Usage

In [12]:
from langchain_community.tools.oxylabs_search import OxylabsSearchResults, OxylabsSearchRun
from langchain_community.utilities import OxylabsSearchAPIWrapper

api_wrapper = OxylabsSearchAPIWrapper()

# Tool `OxylabsSearchRun`

In [13]:
tool_run = OxylabsSearchRun(wrapper=api_wrapper)

In [14]:
response_run = tool_run.invoke({
    "query": "Visit restaurants in Vilnius.",
    "geo_location": "Vilnius,Lithuania",    
})

pprint(response_run)

('  ORGANIC RESULTS ITEMS: \n'
 '    ORGANIC-ITEM-1: \n'
 '    POS: 1\n'
 '    URL: '
 'https://www.tripadvisor.com/Restaurants-g274951-Vilnius_Vilnius_County.html\n'
 '    DESC: Some of the best restaurants in Vilnius for families with children '
 'include: Momo grill Vilnius · Jurgis ir Drakonas Ogmios · RoseHip Vegan\xa0'
 '...\n'
 '    TITLE: THE 10 BEST Restaurants in Vilnius (Updated November ...\n'
 '    SITELINKS: \n'
 '      SITELINKS: \n'
 '      INLINE ITEMS: \n'
 '        INLINE-ITEM-1: \n'
 '        URL: '
 'https://www.tripadvisor.com/Restaurants-g274951-zfp58-Vilnius_Vilnius_County.html\n'
 '        TITLE: Vilnius Dinner Restaurants\n'
 '        INLINE-ITEM-2: \n'
 '        URL: '
 'https://www.tripadvisor.com/Restaurants-g274951-zfp2-Vilnius_Vilnius_County.html\n'
 '        TITLE: Vilnius Breakfast Restaurants\n'
 '        INLINE-ITEM-3: \n'
 '        URL: '
 'https://www.tripadvisor.com/Restaurants-g274951-c8-Vilnius_Vilnius_County.html\n'
 '        TITLE: Cafés in Vil

In [15]:
tool_run = OxylabsSearchRun(
    wrapper=api_wrapper,
    kwargs={
        "result_categories": [
            "local_information",  
            "combined_search_result",
        ]
    }
)

In [16]:
response_run = tool_run.invoke({
    "query": "Visit restaurants in Vilnius.",
    "geo_location": "Vilnius,Lithuania",    
})

pprint(response_run)

('  LOCAL PACK: \n'
 '  ITEMS ITEMS: \n'
 '    ITEMS-ITEM-1: \n'
 '    CID: 13950149882539119249\n'
 '    POS: 1\n'
 '    TITLE: Etno Dvaras\n'
 '    RATING: 4.5\n'
 '    ADDRESS: Lithuanian\n'
 '    ITEMS-ITEM-2: \n'
 '    CID: 711702509070991018\n'
 '    POS: 2\n'
 '    TITLE: Lokys\n'
 '    RATING: 4.5\n'
 '    ADDRESS: Lithuanian\n'
 '    ITEMS-ITEM-3: \n'
 '    CID: 7630589473191639738\n'
 '    POS: 3\n'
 '    TITLE: Senoji trobelė\n'
 '    RATING: 4.4\n'
 '    ADDRESS: Lithuanian\n'
 '\n'
 '  ORGANIC RESULTS ITEMS: \n'
 '    ORGANIC-ITEM-1: \n'
 '    POS: 1\n'
 '    URL: '
 'https://www.tripadvisor.com/Restaurants-g274951-Vilnius_Vilnius_County.html\n'
 '    DESC: Some of the best restaurants in Vilnius for families with children '
 'include: Momo grill Vilnius · Jurgis ir Drakonas Ogmios · RoseHip Vegan\xa0'
 '...\n'
 '    TITLE: THE 10 BEST Restaurants in Vilnius (Updated November ...\n'
 '    SITELINKS: \n'
 '      SITELINKS: \n'
 '      INLINE ITEMS: \n'
 '        INLINE-ITEM

In [17]:
tool_results = OxylabsSearchResults(wrapper=api_wrapper)

In [18]:
import json

# .invoke wraps utility.results
response_results = tool_results.invoke({
    "query": "What is the weather in Shanghai?",
    "geo_location": "China",  
})
response_results = json.loads(response_results)
for item in response_results:
    print(item)

{'paid': [], 'organic': [{'pos': 1, 'url': 'https://www.accuweather.com/en/cn/shanghai/106577/weather-forecast/106577', 'desc': 'Hourly Weather · 1 AM 59°. rain drop 7% · 2 AM 58°. rain drop 7% · 3 AM 58°. rain drop 7% · 4 AM 58°. rain drop 7% · 5 AM 57°. rain drop 7% · 6 AM 55°. rain\xa0...', 'title': 'Shanghai, Shanghai, China Weather Forecast', 'sitelinks': {'inline': [{'url': 'https://www.accuweather.com/en/cn/shanghai/106577/daily-weather-forecast/106577', 'title': 'Daily'}, {'url': 'https://www.accuweather.com/en/cn/shanghai/106577/current-weather/106577', 'title': 'Current Weather'}, {'url': 'https://www.accuweather.com/en/cn/shanghai/106577/hourly-weather-forecast/106577', 'title': 'Hourly'}, {'url': 'https://www.accuweather.com/en/cn/shanghai/106577/october-weather/106577', 'title': 'Monthly'}]}, 'url_shown': 'https://www.accuweather.com› shanghai › weather-for...', 'pos_overall': 1, 'favicon_text': 'AccuWeather'}, {'pos': 2, 'url': 'https://weather.com/weather/tenday/l/7f1418

# Chaining

In [19]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Import necessary modules
import getpass
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI  # Use OpenAIChat instead of AzureChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Define tools and agent setup
tool = OxylabsSearchRun(wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

/home/ausra/Data/AMV-workspace/projects_review/Oxylabs/langchain-project/.venv/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'latest Burning Man floods news'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://www.the-independent.com/topic/burning-man
    DESC: Burning Man attendees face raging dust storm and wait hours to leave · Death of woman on 1st day of Burning Man festival under investigation · Police ...
    TITLE: Burning Man - latest news, breaking stories and comment
    URL_SHOWN: https://www.the-independent.com› topic › burning-man
    FAVICON_TEXT: The Independent
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331
    DESC: Sep 4, 2023 — Tens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.
    TITLE: Burning Man flooding: What happened to stranded ...
    URL_SHOWN: https://abcnews.go.com› story
    FAVICON_TEXT: ABC News - Breaking News, L

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In the latest Burning Man floods, tens of thousands of attendees were stranded at the festival after a downpour and massive flooding occurred over the weekend. People faced challenges due to the flooding and had to wait for hours to leave the festival. To learn more details about the incident, you can visit [ABC News](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331) or [Outside Online](https://www.outsideonline.com/outdoor-adventure/exploration-survival/burning-man-2023-flooding-rain).

> Finished chain.


{'input': 'What happened in the latest Burning Man floods?',
 'output': 'In the latest Burning Man floods, tens of thousands of attendees were stranded at the festival after a downpour and massive flooding occurred over the weekend. People faced challenges due to the flooding and had to wait for hours to leave the festival. To learn more details about the incident, you can visit [ABC News](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331) or [Outside Online](https://www.outsideonline.com/outdoor-adventure/exploration-survival/burning-man-2023-flooding-rain).'}

In [21]:
agent_executor.invoke({"input": "Who won latest 2024 Lithuanian elections and why, please make an politological analysis essay from search results."})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': '2024 Lithuanian elections winner', 'geo_location': 'Lithuania'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election
    DESC: Presidential elections were held in Lithuania on 12 May 2024, alongside a referendum on allowing multiple citizenships. Incumbent President Gitanas Nausėda ...
    TITLE: 2024 Lithuanian presidential election
    SITELINKS: 
      SITELINKS: 
      INLINE ITEMS: 
        INLINE-ITEM-1: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Electoral_system
        TITLE: Electoral system
        INLINE-ITEM-2: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Candidates
        TITLE: Candidates
        INLINE-ITEM-3: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Campaign
        TITLE: Campaign
        INLINE-ITEM-4: 
        URL: https://

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': '2024 Lithuanian elections winner political analysis', 'geo_location': 'Lithuania'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://www.reuters.com/world/europe/lithuania-opposition-seeks-cement-election-win-run-off-votes-2024-10-27/
    DESC: 5 days ago — The opposition Social Democrats claimed victory in Lithuania's parliamentary election on Sunday, which was dominated by frustration with the ...
    TITLE: Lithuania leftist opposition win election, eyes coalition
    URL_SHOWN: https://www.reuters.com› world › europe › lithuania-o...
    FAVICON_TEXT: Reuters
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://www.politico.eu/article/lsdp-lithuania-social-democrat-parliament-election-ingrida-simonyte/
    DESC: 6 days ago — Opposition victory is unlikely to change the country's pro-West and anti-Russia direction, however.
    TITLE: Lithuania's Social Democrats win parliamentary election
    URL_SHOWN: https://ww

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### Analysis of the 2024 Lithuanian Elections

The recent Lithuanian elections in 2024 saw a significant political shift with the opposition Social Democrats claiming victory in both the parliamentary and presidential elections. This victory marked a turning point in Lithuanian politics, indicating a potential change in the direction of the country's governance.

#### Victory of the Social Democrats
The Social Democrats' triumph in the parliamentary election was fueled by widespread frustration with the previous government, particularly concerning issues related to economic policies, social welfare, and governance. The ruling conservatives conceded defeat, paving the way for the Social Democrats to lead the government. This victory was a clear mandate from the Lithuanian electorate for change and a departure from the policies of the previous administration.

#### Implications of the Election Results
The victory of the Social Democrats is expected to have far-reaching implications for L

{'input': 'Who won latest 2024 Lithuanian elections and why, please make an politological analysis essay from search results.',
 'output': "### Analysis of the 2024 Lithuanian Elections\n\nThe recent Lithuanian elections in 2024 saw a significant political shift with the opposition Social Democrats claiming victory in both the parliamentary and presidential elections. This victory marked a turning point in Lithuanian politics, indicating a potential change in the direction of the country's governance.\n\n#### Victory of the Social Democrats\nThe Social Democrats' triumph in the parliamentary election was fueled by widespread frustration with the previous government, particularly concerning issues related to economic policies, social welfare, and governance. The ruling conservatives conceded defeat, paving the way for the Social Democrats to lead the government. This victory was a clear mandate from the Lithuanian electorate for change and a departure from the policies of the previous a

In [22]:
agent_executor.invoke({"input": "What is the most profitable company in lithuania in 2024?"})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'most profitable company in Lithuania in 2024'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://ceoworld.biz/2024/02/19/lithuanias-largest-companies-by-market-capitalization-2024/
    DESC: Feb 19, 2024 — As of October 24, 2024, AB IGNITIS GRUPE is the most valuable company in Lithuania, with a market cap of $1.51 billion. Following it are TELIA ...
    TITLE: Lithuania's Largest Companies by market capitalization, ...
    URL_SHOWN: https://ceoworld.biz› Latest › Banking and Finance
    FAVICON_TEXT: CEOWORLD magazine
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://lithuania.lt/news/business-and-innovations-in-lithuania/which-startups-will-be-making-waves-in-lithuania-in-2024/
    DESC: Jan 23, 2024 — Which startups will be making waves in Lithuania in 2024? · Which startups are worth keeping an eye on in 2024? · 1. Cast AI: · 2. Amlyze: · 3.
    TITLE: Which startups will be making waves in Lithuania in 2024?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE with a market capitalization of $1.51 billion as of October 24, 2024.

> Finished chain.


{'input': 'What is the most profitable company in lithuania in 2024?',
 'output': 'The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE with a market capitalization of $1.51 billion as of October 24, 2024.'}